In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()
from pyspark.sql.functions import isnan, when, count, col

In [2]:
df = spark.read.csv('energydata_complete.csv',inferSchema=True, header=True)
df.printSchema()



root
 |-- date: timestamp (nullable = true)
 |-- Appliances: double (nullable = true)
 |-- lights: double (nullable = true)
 |-- T1: double (nullable = true)
 |-- RH_1: double (nullable = true)
 |-- T2: double (nullable = true)
 |-- RH_2: double (nullable = true)
 |-- T3: double (nullable = true)
 |-- RH_3: double (nullable = true)
 |-- T4: double (nullable = true)
 |-- RH_4: double (nullable = true)
 |-- T5: double (nullable = true)
 |-- RH_5: double (nullable = true)
 |-- T6: double (nullable = true)
 |-- RH_6: double (nullable = true)
 |-- T7: double (nullable = true)
 |-- RH_7: double (nullable = true)
 |-- T8: double (nullable = true)
 |-- RH_8: double (nullable = true)
 |-- T9: double (nullable = true)
 |-- RH_9: double (nullable = true)
 |-- T_out: double (nullable = true)
 |-- Press_mm_hg: double (nullable = true)
 |-- RH_out: double (nullable = true)
 |-- Windspeed: double (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- Tdewpoint: double (nullable = true)
 |--

In [3]:
df.show(1)

+--------------------+----------+------+-----+----------------+----+-----+-----+-----+----+----------------+----------------+----+----------------+----------------+----+----------------+----+----+----------------+-----+-----+-----------+------+---------+----------+---------+------------------+------------------+
|                date|Appliances|lights|   T1|            RH_1|  T2| RH_2|   T3| RH_3|  T4|            RH_4|              T5|RH_5|              T6|            RH_6|  T7|            RH_7|  T8|RH_8|              T9| RH_9|T_out|Press_mm_hg|RH_out|Windspeed|Visibility|Tdewpoint|               rv1|               rv2|
+--------------------+----------+------+-----+----------------+----+-----+-----+-----+----+----------------+----------------+----+----------------+----------------+----+----------------+----+----+----------------+-----+-----+-----------+------+---------+----------+---------+------------------+------------------+
|2016-01-11 17:00:...|      60.0|  30.0|19.89|47.596666666

In [4]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+
|summary|        Appliances|            lights|                T1|              RH_1|                T2|             RH_2|                T3|             RH_3|                T4|             RH_4|                T5|             RH_5|               T6|              RH_6|                T7|             RH_7|                T8|              RH_8|                T9|              RH_9|             T_out|       Press_mm_hg|           RH_out|         Windspeed|    

In [5]:
df.head(1)

[Row(date=datetime.datetime(2016, 1, 11, 17, 0), Appliances=60.0, lights=30.0, T1=19.89, RH_1=47.5966666666667, T2=19.2, RH_2=44.79, T3=19.79, RH_3=44.73, T4=19.0, RH_4=45.5666666666667, T5=17.1666666666667, RH_5=55.2, T6=7.02666666666667, RH_6=84.2566666666667, T7=17.2, RH_7=41.6266666666667, T8=18.2, RH_8=48.9, T9=17.0333333333333, RH_9=45.53, T_out=6.6, Press_mm_hg=733.5, RH_out=92.0, Windspeed=7.0, Visibility=63.0, Tdewpoint=5.3, rv1=13.275433157104999, rv2=13.275433157104999)]

In [6]:
row_number = df.count()
column_number = len(df.dtypes)

In [7]:
print((df.count(), len(df.columns)))


(19735, 29)


In [8]:
df.columns

['date',
 'Appliances',
 'lights',
 'T1',
 'RH_1',
 'T2',
 'RH_2',
 'T3',
 'RH_3',
 'T4',
 'RH_4',
 'T5',
 'RH_5',
 'T6',
 'RH_6',
 'T7',
 'RH_7',
 'T8',
 'RH_8',
 'T9',
 'RH_9',
 'T_out',
 'Press_mm_hg',
 'RH_out',
 'Windspeed',
 'Visibility',
 'Tdewpoint',
 'rv1',
 'rv2']

In [9]:
  df.select("date", "Appliances","lights","Windspeed","Visibility","Press_mm_hg","Tdewpoint").describe().show()


+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|        Appliances|            lights|         Windspeed|       Visibility|       Press_mm_hg|         Tdewpoint|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|             19735|             19735|             19735|            19735|             19735|             19735|
|   mean|  97.6949581960983|3.8018748416518875|4.0397517101596145|38.33083354446415| 755.5226019761848|3.7607068659741527|
| stddev|102.52489053740616| 7.935987600215871|2.4512206057343047|11.79471875736811|7.3994406598535925| 4.194647835033508|
|    min|              10.0|               0.0|               0.0|              1.0|             729.3|              -6.6|
|    max|            1080.0|              70.0|              14.0|             66.0|             772.3|              15.5|
+-------+-------

In [10]:
 df.select("T1", "T2","T3","T4","T5","T6","T7","T8","T9","T_out").describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|                T1|                T2|                T3|                T4|                T5|               T6|                T7|                T8|                T9|             T_out|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|  count|             19735|             19735|             19735|             19735|             19735|            19735|             19735|             19735|             19735|             19735|
|   mean|21.686571386748575|20.341219463849324|22.267610984880132|20.855334722410657| 19.59210632801864|7.910939332403554|20.267106470136902| 22.02910672298018|19.485828160608985|7.4116645553585565|
| std

In [11]:
 df.select("T1", "T2","T3","T4","T5","T6").describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|                T1|                T2|                T3|                T4|                T5|               T6|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|             19735|             19735|             19735|             19735|             19735|            19735|
|   mean|21.686571386748575|20.341219463849324|22.267610984880132|20.855334722410657| 19.59210632801864|7.910939332403554|
| stddev|1.6060656448140267|2.1929735335219647|2.0061105361434204|2.0428844769855043|1.8446232735005959|6.090346609884242|
|    min|             16.79|              16.1|              17.2|              15.1|             15.33|           -6.065|
|    max|             26.26|  29.8566666666667|            29.236|              26.2|            25.795|            28.29|
+-------+-------

In [12]:
 df.select("T7","T8","T9","T_out").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|                T7|                T8|                T9|             T_out|
+-------+------------------+------------------+------------------+------------------+
|  count|             19735|             19735|             19735|             19735|
|   mean|20.267106470136902| 22.02910672298018|19.485828160608985|7.4116645553585565|
| stddev| 2.109993324051789|1.9561617218042135| 2.014712386074535| 5.317408805653679|
|    min|             15.39|  16.3066666666667|             14.89|              -5.0|
|    max|              26.0|             27.23|              24.5|              26.1|
+-------+------------------+------------------+------------------+------------------+



In [13]:
 df.select("RH_1", "RH_2","RH_3","RH_4","RH_5","RH_6").describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+
|summary|              RH_1|             RH_2|             RH_3|             RH_4|             RH_5|              RH_6|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+
|  count|             19735|            19735|            19735|            19735|            19735|             19735|
|   mean|40.259739279784384|40.42042041370675|39.24250007720708|39.02690378814554|50.94928262962212|54.609083387583084|
| stddev| 3.979298830435571|4.069812540449677|3.254576492620434|4.341320653139022|9.022034306523485| 31.14980587551157|
|    min|  27.0233333333333| 20.4633333333333| 28.7666666666667|            27.66|           29.815|               1.0|
|    max|             63.36| 56.0266666666667| 50.1633333333333|            51.09| 96.3216666666667|              99.9|
+-------+------------------+------------

In [14]:
 df.select("RH_7","RH_8","RH_9","RH_out").describe().show()

+-------+-----------------+------------------+------------------+-----------------+
|summary|             RH_7|              RH_8|              RH_9|           RH_out|
+-------+-----------------+------------------+------------------+-----------------+
|  count|            19735|             19735|             19735|            19735|
|   mean|35.38820021508418| 42.93616537238887|41.552400753376595|79.75041803901698|
| stddev|5.114208029896672|5.2243606795975115|   4.1514973238505|14.90108755752694|
|    min|             23.2|              29.6|  29.1666666666667|             24.0|
|    max|             51.4|             58.78|  53.3266666666667|            100.0|
+-------+-----------------+------------------+------------------+-----------------+



In [15]:
 df.select("rv1","rv2").describe().show()

+-------+--------------------+--------------------+
|summary|                 rv1|                 rv2|
+-------+--------------------+--------------------+
|  count|               19735|               19735|
|   mean|  24.988033485049435|  24.988033485049435|
| stddev|  14.496633860415896|  14.496633860415896|
|    min|0.005321681965142...|0.005321681965142...|
|    max|    49.9965296825394|    49.9965296825394|
+-------+--------------------+--------------------+



In [16]:
df1 = df.drop('rv1','rv2','date')

In [17]:
df1.columns

['Appliances',
 'lights',
 'T1',
 'RH_1',
 'T2',
 'RH_2',
 'T3',
 'RH_3',
 'T4',
 'RH_4',
 'T5',
 'RH_5',
 'T6',
 'RH_6',
 'T7',
 'RH_7',
 'T8',
 'RH_8',
 'T9',
 'RH_9',
 'T_out',
 'Press_mm_hg',
 'RH_out',
 'Windspeed',
 'Visibility',
 'Tdewpoint']

In [18]:
# First, we have to register the DataFrame as a SQL temporary view.
df.createOrReplaceTempView('df1')

# After that, we can use the SQL programming language for queries. 
results = spark.sql("SELECT * FROM df1")

In [19]:
from pyspark.sql.functions import col
df1.describe().filter(col("summary") == "count").show()

+-------+----------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+------+---------+----------+---------+
|summary|Appliances|lights|   T1| RH_1|   T2| RH_2|   T3| RH_3|   T4| RH_4|   T5| RH_5|   T6| RH_6|   T7| RH_7|   T8| RH_8|   T9| RH_9|T_out|Press_mm_hg|RH_out|Windspeed|Visibility|Tdewpoint|
+-------+----------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+------+---------+----------+---------+
|  count|     19735| 19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|19735|      19735| 19735|    19735|     19735|    19735|
+-------+----------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+------+---------+----------+---------+



In [20]:
#correlation 'target' 

from pyspark.ml.feature import VectorAssembler


features = df1.rdd.map(lambda row: row[0:])

from pyspark.mllib.stat import Statistics

corr_mat=Statistics.corr(features, method="pearson")

#corr_mat.show()

#from pyspark.mllib.stat import Statistics
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("check").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

#arr = np.array([[2,3], [2,8], [2,3],[4,5]])

#from pyspark import SparkContext
#sc = SparkContext().getOrCreate(conf=conf)

rdd1 = sc.parallelize(corr_mat)
rdd2 = rdd1.map(lambda x: [float(i) for i in x])
dftest= rdd2.toDF(["Target", "2",'3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
dftest.select('Target').show(30)
print(len(dftest.columns))
#SparkContext


+--------------------+
|              Target|
+--------------------+
|                 1.0|
| 0.19727756020624476|
| 0.05544747185169637|
| 0.08603142531162396|
| 0.12007328289784344|
|-0.06046476002691181|
| 0.08505990236538934|
|  0.0362917211535374|
| 0.04028098301073038|
|0.016965431418945142|
|0.019759860588529344|
|0.006954536262407...|
| 0.11763768942239645|
|-0.08317760704201052|
| 0.02580104461954486|
|-0.05564241833638...|
|0.039571981095422364|
|-0.09403862707400816|
| 0.01001028464454249|
|-0.05146201134629927|
| 0.09915465073996763|
|-0.03488482907924371|
| -0.1522823809092063|
| 0.08712210591236885|
|2.303578554977464E-4|
|0.015353010487474785|
+--------------------+

26


In [29]:
from pyspark.sql.functions import col, floor, lit, concat
df = df.withColumn('Appliances',concat(lit('group'),(1+floor(col('Appliances')/3))))
df.show()

+--------------------+----------+------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+------------------+------------------+
|                date|Appliances|lights|              T1|            RH_1|              T2|            RH_2|              T3|            RH_3|              T4|            RH_4|              T5|            RH_5|              T6|            RH_6|              T7|            RH_7|              T8|            RH_8|              T9|            RH_9|           T_out|     Press_mm_hg|          RH_out|       Windspeed|      Visibility|       Tdewpoint|               rv1|               rv2|
+--------------------+----

In [32]:
df.select("Appliances").describe().show()

+-------+----------+
|summary|Appliances|
+-------+----------+
|  count|         0|
|   mean|      null|
| stddev|      null|
|    min|      null|
|    max|      null|
+-------+----------+

